# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import pickle

import re

import nltk 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # defining a url pattern with regex
    url_pattern = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Detect and replace urls
    detected_urls = re.findall(url_pattern, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    # tokenize sentences
    tokens = word_tokenize(text)
    
    # iniciate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # save cleaned tokens
    cleaned_tokens = [lemmatizer.lemmatize(token).lower().strip() for token in tokens]
    
    # remove stopwords
    english_stopwords = list(set(stopwords.words('english')))
    cleaned_tokens = [token for token in cleaned_tokens if token not in english_stopwords]
    
    return cleaned_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
def create_pipeline():
    return Pipeline([
        ('vec', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, n_jobs=6)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)
pipeline = create_pipeline()


pipeline.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def create_analyse(pipeline, X, Y):
    # predict on the X_test
    y_pred = pipeline.predict(X)
    
    # build classification report on every column
    performances = []
    for i in range(len(Y.columns)):
        performances.append([f1_score(Y.iloc[:, i].values, y_pred[:, i], average='micro'),
                             precision_score(Y.iloc[:, i].values, y_pred[:, i], average='micro'),
                             recall_score(Y.iloc[:, i].values, y_pred[:, i], average='micro')])
    # build dataframe
    performances = pd.DataFrame(performances, columns=['f1 score', 'precision', 'recall'],
                                index = Y.columns)   
    return performances


In [7]:
create_analyse(pipeline, x_test, y_test)

,f1 score,precision,recall
related,0.812023,0.812023,0.812023
request,0.888160,0.888160,0.888160
offer,0.995117,0.995117,0.995117
aid_related,0.772963,0.772963,0.772963
medical_help,0.919896,0.919896,0.919896
medical_products,0.951022,0.951022,0.951022
search_and_rescue,0.974214,0.974214,0.974214
security,0.983674,0.983674,0.983674
military,0.969789,0.969789,0.969789
child_alone,1.000000,1.000000,1.000000



### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'clf__estimator__n_estimators': [50, 100]}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters)
cv.fit(x_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
create_analyse(cv, x_test, y_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
improved_pipeline = Pipeline([
                                ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer()),
                                ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=100)))
                            ])

improved_pipeline.fit(x_train, y_train)
y_pred_improved = improved_pipeline.predict(x_test)
create_analyse(improved_pipeline, x_test, y_test)

### 9. Export your model as a pickle file

In [8]:
pickle.dump(pipeline, open('test-model.pkl', 'wb'))
# pickle.dump(improved_pipeline, open('improved_model.pkl', 'wb'))


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.